In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# carregar o dataset
df = pd.read_csv("df_treino.csv")  

In [3]:
# define uma funcao para limpar os textos
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Substitui múltiplos espaços por um único
    text = re.sub(r'[^a-záéíóúãõâêîôûàèìòùç]+', ' ', text)  # Remove caracteres especiais e números
    text = text.strip()
    return text


In [4]:
# Exemplo de como fazer previsões
def predict_text(texto):
    cleaned_text = clean_text(texto)
    prediction = pipeline.predict([cleaned_text])
    predicted_class = label_encoder.inverse_transform(prediction)
    return predicted_class[0]

In [5]:
# Aplicando a limpeza no reviewBody
df['cleaned_review'] = df['reviewBody'].apply(clean_text)

In [6]:
# Codificando as classes (por exemplo, fake, relevante, irrelevante)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['alternativeName'])

In [7]:
# Divisão do dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['label'], test_size=0.2, random_state=42)


In [8]:
# Carregar stopwords em português do NLTK
nltk.download('stopwords')
from nltk.corpus import stopwords

# Obter as stopwords em português
stopwords_pt = stopwords.words('portuguese')

# Criar o modelo com TfidfVectorizer usando stopwords em português
pipeline = make_pipeline(
    TfidfVectorizer(stop_words=stopwords_pt, ngram_range=(1, 2), max_features=5000),
    MultinomialNB()
)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wectornanime\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Treinando o modelo
pipeline.fit(X_train, y_train)


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...])),
                ('multinomialnb', MultinomialNB())])

In [10]:
# Previsão no conjunto de teste
y_pred = pipeline.predict(X_test)


In [11]:
# # Avaliação do modelo
# print("Acurácia:", accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [12]:
# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))

# Verificar classes presentes em y_test e y_pred
print("Classes no conjunto de teste:", np.unique(y_test))
print("Classes nas previsões:", np.unique(y_pred))

# Para garantir que todas as classes sejam incluídas no classification_report, use labels explicitamente
# Vamos usar np.unique de y_train para garantir que todas as classes que o modelo pode prever sejam consideradas

# Ajustando target_names para usar as classes presentes no label_encoder
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, labels=np.unique(y_train)))

Acurácia: 0.90625
Classes no conjunto de teste: [0 1]
Classes nas previsões: [0 1]
              precision    recall  f1-score   support

       Falso       0.93      0.88      0.90        16
  Verdadeiro       0.88      0.94      0.91        16

    accuracy                           0.91        32
   macro avg       0.91      0.91      0.91        32
weighted avg       0.91      0.91      0.91        32



In [15]:
# Teste com exemplo
texto_exemplo = "O deputado federal Silvio Torres PSDB-SP, secretário-geral do partido, durante discurso feito no plenário da Câmara no dia 14 de junho, disse que pode haver três anos de queda do PIB, algo nunca antes experimentado em nossa economia."
predicao = predict_text(texto_exemplo)
print(f"A classificação do texto é: {predicao}")

A classificação do texto é: Verdadeiro
